In [17]:
#Import libraries
import pandas as pd
import numpy as np
import glob
import os

In [18]:
#Combine the files into one
#Make sure the foler only has the CSV files you want to combine

#Define the path
path = r'C:\Users\billf\Desktop\Data_Science\FirstOfficialMLProject\School_stats_scraped_files'

#Get all the CSV files into an array
all_files = glob.glob(os.path.join(path, "*.csv"))

#Turn each csv file into a DF
df_from_each_file = (pd.read_csv(f) for f in all_files)

#Concat them together
concatenated_ratings = pd.concat(df_from_each_file, ignore_index=True,sort=False)

In [19]:
#Do the same for results
path = r'C:\Users\billf\Desktop\Data_Science\FirstOfficialMLProject\Spread_and_results_scraped_files'
all_files = glob.glob(os.path.join(path, "*.csv"))
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_results = pd.concat(df_from_each_file, ignore_index=True, sort=False)

In [20]:
#Save results into CSV files
concatenated_ratings.to_csv("School_stats_files/school_stats_2010_to_2017.csv")
concatenated_results.to_csv("Spread_and_results_files/spread_score_results_2010_to_2017.csv")

In [21]:
#Define the DF to work with
ratings = pd.read_csv("School_stats_files/school_stats_2010_to_2017.csv")
#ratings

In [22]:
#Drop the first two columns which are weird unnamed columns
ratings.drop(ratings.columns[[0,1]], axis=1, inplace=True)

In [23]:
#Replace spacing between school names to help with formatting
ratings["School"] = ratings["School"].str.replace(' ', '')

In [24]:
#Since our school names come from two different sources, need to standardize the names
ratings['School'] = ratings.replace({'School' : {'AppalachianState':'AppalachianSt',
                             'BowlingGreenState':'BowlingGreen',
                             'BrighamYoung':'BYU',
                             'Buffalo': 'BuffaloU',
                             'FloridaInternational': 'FloridaIntl',
                             'Houston':'HoustonU',
                             'Louisiana':'ULLafayette',
                             'Louisiana-Monroe':'ULMonroe',
                             'Miami(FL)':'MiamiFlorida',
                             'Miami(OH)':'MiamiOhio',
                             'MiddleTennesseeState':'MiddleTennSt',
                             'Nevada-LasVegas':'UNLV',
                             'NorthCarolinaState':'NCState',
                             'NorthernIllinois':'NoIllinois',
                             'OleMiss':'Mississippi',
                             'Pitt':'PittsburghU',
                             'SouthernMississippi':'SouthernMiss',
                             'TexasChristian':'TCU',
                             'UCF':'CentralFlorida',
                             'UTSA':'TexSanAntonio',
                             'Arizona':'ArizonaU',
                             'Cincinnati':'CincinnatiU',
                             'Minnesota':'MinnesotaU',
                             'MississippiState':'MississippiSt',
                             'Tennessee':'TennesseeU',
                             'Washington':'WashingtonU'
                            }})

In [25]:
#Pull in the results df
results = pd.read_csv("Spread_and_results_files/spread_score_results_2010_to_2017.csv")

In [26]:
#Drop first two columns
results.drop(results.columns[[0,1]], axis=1, inplace=True)

In [27]:
#Drop unwanted columnds
results = results.drop(results[["Rot","1st","2nd","3rd","4th","2H"]], axis=1)

In [28]:
#Games are on alternating lines, but we want them to be on the same row
#We will seperate the games into two different dfs, 1 and 2
df1 = results[results.index % 2 != 0]
df2 = results[results.index % 2 == 0]

In [29]:
#Rename columns to avoid confusion
df1.rename(index=str, columns={"Date": "Date1", "VH": "VH1", "Team": "Team1",
                               "Final": "Final1", "Open": "Open1", "Close": "Close1",
                              "ML": "ML1", "year": "Year1"}, inplace = True)
df2.rename(index=str, columns={"Date": "Date2", "VH": "VH2", "Team": "Team2",
                               "Final": "Final2", "Open": "Open2", "Close": "Close2",
                              "ML": "ML2", "year": "Year2"}, inplace = True)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [30]:
#Concat the resulting dfs so that each game is on its own row
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)
results = pd.concat([df1, df2], axis=1, sort=False)

In [31]:
#Drop some values
results = results.drop(results[["Date2","Year1"]],axis=1)

In [32]:
#Rename some values
results = results.rename(index=str, columns={"Date1":"Date", "Year2":"Year","VH1":"Team1Site","VH2":"Team2Site"})

In [33]:
#Some names need renaming again
results = results.replace({'Arizona':'ArizonaU',
                            'Buffalo':'BuffaloU',
                            'Cincinnati':'CincinnatiU',
                            'Houston':'HoustonU',
                            'MidTennSt':'MiddleTennSt',
                            'MidTennesseeState':'MiddleTennSt',
                            'Minnesota':'MinnesotaU',
                            'MississippiState': 'MississippiSt',
                            'Pittsburgh':'PittsburghU',
                            'SoMississippi':'SouthernMiss',
                            'Tennessee':'TennesseeU',
                            'UL-Monroe':'ULMonroe',
                            'Washington':'WashingtonU'
                          })

In [34]:
#Checking to make sure there are no more values that do not match
results_team1s = set(results.Team1.unique())
results_team2s = set(results.Team2.unique())
ratings_teams = set(ratings.School.unique())
#results_team1s
#results_team2s
#ratings_teams

In [35]:
#Checking to make sure there are no more values that do not match
Teams_in_both = list(results_team1s.intersection(ratings_teams))
Teams_in_both

['Northwestern',
 'OhioState',
 'Navy',
 'IowaState',
 'Memphis',
 'Hawaii',
 'ColoradoState',
 'Virginia',
 'WakeForest',
 'TennesseeU',
 'Wisconsin',
 'Baylor',
 'CentralMichigan',
 'EastCarolina',
 'OldDominion',
 'BallState',
 'NCState',
 'Illinois',
 'SanJoseState',
 'WesternKentucky',
 'BostonCollege',
 'FloridaIntl',
 'MiamiFlorida',
 'Vanderbilt',
 'VirginiaTech',
 'Syracuse',
 'Temple',
 'WestVirginia',
 'HoustonU',
 'FresnoState',
 'WashingtonU',
 'CentralFlorida',
 'PittsburghU',
 'Massachusetts',
 'LouisianaTech',
 'MichiganState',
 'TexasState',
 'FloridaState',
 'Connecticut',
 'Mississippi',
 'Marshall',
 'CoastalCarolina',
 'KentState',
 'Duke',
 'Rice',
 'GeorgiaTech',
 'Akron',
 'Nebraska',
 'Missouri',
 'BowlingGreen',
 'LSU',
 'SouthFlorida',
 'FloridaAtlantic',
 'Florida',
 'NorthCarolina',
 'UCLA',
 'BoiseState',
 'WashingtonState',
 'MiddleTennSt',
 'ULMonroe',
 'NorthTexas',
 'SanDiegoState',
 'Arkansas',
 'Kansas',
 'BuffaloU',
 'NewMexicoState',
 'NotreDame',


In [36]:
#Checking to make sure there are no more values that do not match
unique_results_team1s = list(set(results_team1s)-set(Teams_in_both))
unique_results_team2s = list(set(results_team2s)-set(Teams_in_both))
unique_ratings_teams = list(set(ratings_teams)-set(Teams_in_both))

In [37]:
#Array should be empty
unique_ratings_teams.sort()
unique_ratings_teams

[]

In [38]:
unique_results_team1s.sort()

In [39]:
unique_results_team2s.sort()

In [40]:
#Save to CSV files
results.to_csv("Spread_and_results_files/processed_results.csv")
ratings.to_csv("School_stats_files/processed_ratings.csv")